# Import

In [1]:
# ============================================================


!apt install tesseract-ocr -y -qq
!apt install libtesseract-dev -y -qq
!pip install Pillow -q
!pip install --upgrade transformers -q
!pip install pytesseract -q
!sudo apt install tesseract-ocr-vie -y -qq
!sudo apt-get install tesseract-ocr-all -y -qq
!pip install flask flask-cors pyngrok -q
!pip install craft-detector==0.2.1 -q


tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Selecting previously unselected package libleptonica-dev.
Preparing to unpack .../libleptonica-dev_1.82.0-3build1_amd64.deb ...
Unpacking libleptonica-dev (1.82.0-3build1) ...
Selecting previously unselected package libtesseract-dev:amd64.
Preparing to unpack ...

# LOAD  models

In [ ]:

import time
start_time = time.time()

# ==================== MOUNT DRIVE ====================
from google.colab import drive
drive.mount('/content/drive')

# ==================== IMPORT THƯ VIỆN ====================
print("\n [1/4] Đang import thư viện...")

import cv2
import numpy as np
import pytesseract
from pytesseract import Output
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from collections import OrderedDict
import json
import base64
import threading
import time as time_module

# Flask
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

print(" Import thư viện thành công!")

# ==================== CONFIG ====================
pytesseract.pytesseract.tesseract_cmd = r'/bin/tesseract'

# Đường dẫn models - THAY ĐỔI NẾU CẦN
CRAFT_MODEL_PATH = '/content/drive/MyDrive/craft_mlt_25k.pth'
REFINER_MODEL_PATH = '/content/drive/MyDrive/craft_refiner_CTW1500.pth'

# CRAFT config
TEXT_THRESHOLD = 0.8
LINK_THRESHOLD = 0.7
LOW_TEXT = 0.4
POLY = False
SET_REFINE = False

# ==================== LOAD LLaMA MODEL ====================
print("\n [2/4] Đang load LLaMA model... (2-5 phút)")

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

#  THAY TOKEN CỦA BẠN
HF_TOKEN = '........'
login(token=HF_TOKEN, new_session=True)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
llama_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Set pad token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(" LLaMA model đã load xong!")

# ==================== LOAD CRAFT MODEL ====================
print("\n [3/4] Đang load CRAFT model...")

from craft_detector import craft_utils, imgproc, file_utils
from craft_detector.craft import CRAFT

def copyStateDict(state_dict):
    if list(state_dict.keys())[0].startswith("module"):
        start_idx = 1
    else:
        start_idx = 0
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = ".".join(k.split(".")[start_idx:])
        new_state_dict[name] = v
    return new_state_dict

# Load CRAFT
craft_net = CRAFT()
if torch.cuda.is_available():
    craft_net.load_state_dict(copyStateDict(torch.load(CRAFT_MODEL_PATH)))
    craft_net = craft_net.cuda()
    craft_net = torch.nn.DataParallel(craft_net)
    cudnn.benchmark = False
else:
    craft_net.load_state_dict(copyStateDict(torch.load(CRAFT_MODEL_PATH, map_location='cpu')))
craft_net.eval()

# Refiner (optional)
refine_net = None
if SET_REFINE:
    from craft_detector.refinenet import RefineNet
    refine_net = RefineNet()
    if torch.cuda.is_available():
        refine_net.load_state_dict(copyStateDict(torch.load(REFINER_MODEL_PATH)))
        refine_net = refine_net.cuda()
        refine_net = torch.nn.DataParallel(refine_net)
    else:
        refine_net.load_state_dict(copyStateDict(torch.load(REFINER_MODEL_PATH, map_location='cpu')))
    refine_net.eval()

print(" CRAFT model đã load xong!")

# ==================== WARMUP MODELS ====================
print("\n [4/4] Đang warmup models...")

# Warmup LLaMA
warmup_text = "Test receipt"
warmup_messages = [
    {"role": "system", "content": "Extract JSON from text."},
    {"role": "user", "content": warmup_text}
]
warmup_ids = tokenizer.apply_chat_template(
    warmup_messages, add_generation_prompt=True, return_tensors="pt"
).to(llama_model.device)

with torch.no_grad():
    _ = llama_model.generate(warmup_ids, max_new_tokens=10, do_sample=False)
print("  ✓ LLaMA warmup done")

# Warmup CRAFT
dummy_image = np.zeros((100, 100, 3), dtype=np.uint8)
x = imgproc.normalizeMeanVariance(dummy_image)
x = torch.from_numpy(x).permute(2, 0, 1).unsqueeze(0)
if torch.cuda.is_available():
    x = x.cuda()
with torch.no_grad():
    _ = craft_net(x)
print("  ✓ CRAFT warmup done")

# Warmup Tesseract
_ = pytesseract.image_to_string(dummy_image, lang="vie")
print("  ✓ Tesseract warmup done")

# ==================== HOÀN TẤT ====================
elapsed = time.time() - start_time
print("\n" + "=" * 60)
print(f" CELL 2 HOÀN TẤT - Tất cả models đã sẵn sàng!")
print(f"  Thời gian load: {elapsed:.1f} giây")
print("=" * 60)
print(" Tiếp tục chạy CELL 3...")

# Flag để đánh dấu models đã load
MODELS_READY = True

Mounted at /content/drive

 [1/4] Đang import thư viện...
 Import thư viện thành công!

 [2/4] Đang load LLaMA model... (2-5 phút)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

 LLaMA model đã load xong!

 [3/4] Đang load CRAFT model...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 CRAFT model đã load xong!

 [4/4] Đang warmup models...
  ✓ LLaMA warmup done
  ✓ CRAFT warmup done
  ✓ Tesseract warmup done

 CELL 2 HOÀN TẤT - Tất cả models đã sẵn sàng!
  Thời gian load: 138.3 giây
 Tiếp tục chạy CELL 3...


# Def

In [3]:



print(" Đang định nghĩa các hàm xử lý...")

# ==================== IMAGE PROCESSING ====================

def rotate_image(gray_img):
    """Xoay ảnh để text nằm ngang"""
    try:
        imOTSU = cv2.threshold(gray_img, 0, 1, cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)[1]
        coords = np.column_stack(np.where(imOTSU > 0))
        if len(coords) == 0:
            h, w = gray_img.shape
            return np.float32([[1, 0, 0], [0, 1, 0]]), h, w
        angle = cv2.minAreaRect(coords)[-1]
        if angle < -45:
            angle = -(90 + angle)
        else:
            angle = -angle
        h, w = gray_img.shape
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return M, h, w
    except:
        h, w = gray_img.shape
        return np.float32([[1, 0, 0], [0, 1, 0]]), h, w

def auto_morphology(thresh):
    """Tự động chọn kernel size"""
    text_pixels = cv2.countNonZero(thresh)
    total_pixels = thresh.shape[0] * thresh.shape[1]
    density = text_pixels / total_pixels
    if density > 0.15:
        ksize = (1, 1)
    elif density > 0.03:
        ksize = (3, 3)
    elif density > 0.01:
        ksize = (5, 5)
    else:
        ksize = (7, 7)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize)
    dilated = cv2.dilate(thresh, kernel, iterations=2)
    closed = cv2.erode(dilated, kernel, iterations=2)
    return closed

def preprocess_pipeline(image):
    """Pipeline tiền xử lý ảnh"""
    if len(image.shape) == 2:
        gray = image
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.4, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    background = cv2.GaussianBlur(gray, (55, 55), 0)
    flattened = cv2.divide(gray, background, scale=255)
    thresh = cv2.threshold(flattened, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    closed = auto_morphology(thresh)
    return closed

# ==================== CRAFT DETECTION ====================

def test_net(net, image, text_threshold, link_threshold, low_text, cuda, poly, refine_net=None):
    """CRAFT text detection - Đã được tối ưu"""
    high, wide = image.shape[:2]
    img_resized, target_ratio = imgproc.resize_aspect_ratio(
        image, target_size=max(high, wide),
        interpolation=cv2.INTER_LINEAR, mag_ratio=1.5
    )
    ratio_h = ratio_w = 1 / target_ratio

    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))
    if cuda:
        x = x.cuda()

    with torch.no_grad():
        y, feature = net(x)

    score_text = y[0,:,:,0].cpu().data.numpy()
    score_link = y[0,:,:,1].cpu().data.numpy()

    if refine_net is not None:
        with torch.no_grad():
            y_refiner = refine_net(y, feature)
        score_link = y_refiner[0,:,:,0].cpu().data.numpy()

    boxes, polys, mapper = craft_utils.getDetBoxes(
        score_text, score_link, text_threshold, link_threshold, low_text, poly
    )

    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    polys = craft_utils.adjustResultCoordinates(polys, ratio_w, ratio_h)
    for k in range(len(polys)):
        if polys[k] is None:
            polys[k] = boxes[k]

    return boxes, polys

def loadImage(img_file):
    """Load và chuẩn hóa ảnh"""
    img = img_file
    if img.shape[0] == 2:
        img = img[0]
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    if img.shape[2] == 4:
        img = img[:,:,:3]
    return np.array(img)

def detect_text_regions(image):
    """Phát hiện vùng text trong ảnh"""
    coords_list = []
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb = loadImage(image_rgb)
    bboxes, polys = test_net(
        craft_net, image_rgb, TEXT_THRESHOLD, LINK_THRESHOLD,
        LOW_TEXT, torch.cuda.is_available(), POLY, refine_net
    )
    for box in polys:
        poly_arr = np.array(box).astype(np.int32).reshape((-1))
        strResult = ','.join([str(p) for p in poly_arr])
        coords_list.append(strResult)
    return coords_list

def crop_text_regions(image):
    """Cắt các vùng text từ ảnh"""
    lst = detect_text_regions(image)
    crop_img = []
    bboxes = []

    for coord_str in lst:
        coords = [int(x) for x in coord_str.split(',')]
        points = np.array(coords, np.int32).reshape((-1, 1, 2))

        y_min = max(0, points[:,0,1].min())
        y_max = points[:,0,1].max()
        x_min = max(0, points[:,0,0].min())
        x_max = points[:,0,0].max()

        if y_max > y_min and x_max > x_min:
            crop = image[y_min:y_max, x_min:x_max]
            if crop.size > 0:
                preprocess_cropped = preprocess_pipeline(crop)
                crop_img.append(preprocess_cropped)
                bboxes.append({
                    'x_min': int(x_min),
                    'y_min': int(y_min),
                    'x_max': int(x_max),
                    'y_max': int(y_max)
                })

    return crop_img, bboxes

# ==================== LLaMA EXTRACTION ====================

def extract_with_llama(ocr_text: str) -> dict:
    """Trích xuất thông tin bằng LLaMA - Đã được tối ưu"""
    lines = [line.strip() for line in ocr_text.split('\n') if line.strip()]
    text_blob = "\n".join(lines)

    if not text_blob:
        return get_empty_result()

    messages = [
        {
            "role": "system",
            "content": (
                "You are a receipt data extractor. Return ONLY valid JSON with this exact structure: "
                '{"store_name": "string or null", "address": "string or null", "date": "string or null", '
                '"staff_name": "string or null", "staff_id": "string or null", '
                '"line_items": [{"product_name": "string", "quantity": number, "unit_price": number, "total_price": number}], '
                '"change:": float, ' '"total": "string or null"}. '
                "Use null for missing fields. Numbers should be numeric, not strings."
            )
        },
        {
            "role": "user",
            "content": f"Extract data from this receipt:\n{text_blob}"
        }
    ]



    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(llama_model.device)

    with torch.no_grad():
        outputs = llama_model.generate(
            input_ids,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=1024,
            do_sample=False,
            temperature=None,
            top_p=None,
        )

    response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    try:
        # Parse JSON từ response
        clean_str = response.replace("```json", "").replace("```", "").strip()
        start = clean_str.find('{')
        end = clean_str.rfind('}') + 1
        if start != -1 and end > start:
            clean_str = clean_str[start:end]
        return json.loads(clean_str)
    except json.JSONDecodeError:
        print(f" JSON parse error. Raw: {response[:200]}")
        return get_empty_result()

def get_empty_result():
    """Trả về kết quả rỗng"""
    return {
        "store_name": None,
        "address": None,
        "date": None,
        "staff_name": None,
        "staff_id": None,
        "line_items": [],
        "total": None
    }

# ==================== OCR FUNCTIONS ====================

def ocr_full_image(image):
    """OCR toàn bộ ảnh"""
    return pytesseract.image_to_string(image, lang="vie", config='--oem 3 --psm 6')

def ocr_cropped_regions(cropped_images):
    """OCR từng vùng đã crop"""
    all_lines = []
    for crop in cropped_images:
        ocr_data = pytesseract.image_to_data(
            crop, output_type=Output.DICT,
            config='--oem 3 --psm 6', lang="vie"
        )
        line_dict = {}
        for i in range(len(ocr_data['text'])):
            word = ocr_data['text'][i].strip()
            conf = int(ocr_data['conf'][i])
            line_num = ocr_data['line_num'][i]
            if word and conf >= 70:
                if line_num not in line_dict:
                    line_dict[line_num] = []
                line_dict[line_num].append(word)

        for line_words in line_dict.values():
            line = " ".join(line_words)
            if line:
                all_lines.append(line)

    return "\n".join(all_lines)

def count_missing_fields(result):
    """Đếm số trường bị thiếu"""
    count = 0
    fields = ["store_name", "address", "date", "staff_name", "staff_id", "total"]
    for field in fields:
        if not result.get(field):
            count += 1
    if not result.get("line_items"):
        count += 1
    return count

# ==================== MAIN API FUNCTION ====================

def process_image_api(image_bytes):
    """
    ⚡ HÀM XỬ LÝ CHÍNH - ĐÃ TỐI ƯU
    Models đã được load sẵn nên xử lý rất nhanh
    """
    try:
        process_start = time_module.time()

        # 1. Decode image
        nparr = np.frombuffer(image_bytes, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if image is None:
            return {"error": "Cannot decode image"}

        print(f"📷 Image: {image.shape}")

        # 2. OCR toàn bộ ảnh trước (nhanh)
        t1 = time_module.time()
        raw_text = ocr_full_image(image)
        print(f"   OCR full image: {time_module.time()-t1:.2f}s")

        # 3. Extract với LLaMA
        t2 = time_module.time()
        result = extract_with_llama(raw_text)
        print(f"   LLaMA extract: {time_module.time()-t2:.2f}s")

        # 4. Nếu thiếu nhiều thông tin, thử crop và OCR lại
        missing_count = count_missing_fields(result)
        if missing_count > 3:
            print(f"   Missing {missing_count} fields, trying crop method...")
            t3 = time_module.time()
            cropped_images, bboxes = crop_text_regions(image)
            if cropped_images:
                crop_text = ocr_cropped_regions(cropped_images)
                if crop_text:
                    result = extract_with_llama(crop_text)
            print(f"  ⏱️ Crop+OCR: {time_module.time()-t3:.2f}s")
        else:
            # Vẫn detect để lấy bboxes cho visualization
            _, bboxes = crop_text_regions(image)

        # 5. Xác định missing fields
        missing_fields = []
        check_list = ["store_name","address","date","staff_name","staff_id","line_items","total",'change']
        for field in check_list:
            val = result.get(field)
            if val is None or val == "" or val == []:
                missing_fields.append(field)

        # 6. Tạo annotated image
        annotated = image.copy()
        for bbox in bboxes:
            cv2.rectangle(
                annotated,
                (bbox['x_min'], bbox['y_min']),
                (bbox['x_max'], bbox['y_max']),
                (0, 255, 0), 2
            )
        _, buffer = cv2.imencode('.jpg', annotated, [cv2.IMWRITE_JPEG_QUALITY, 85])
        annotated_base64 = base64.b64encode(buffer).decode('utf-8')

        total_time = time_module.time() - process_start
        print(f" Done! Total: {total_time:.2f}s | Missing: {missing_fields}")

        return {
            "extracted_data": result,
            "raw_text": raw_text,
            "missing_fields": missing_fields,
            "bounding_boxes": bboxes,
            "annotated_image": annotated_base64,
            "processing_time": round(total_time, 2)
        }

    except Exception as e:
        import traceback
        traceback.print_exc()
        return {"error": str(e)}

print(" CELL 3 HOÀN TẤT - Các hàm xử lý đã sẵn sàng!")
print(" Tiếp tục chạy CELL 4 để khởi động server...")

 Đang định nghĩa các hàm xử lý...
 CELL 3 HOÀN TẤT - Các hàm xử lý đã sẵn sàng!
 Tiếp tục chạy CELL 4 để khởi động server...


# SERVER

In [ ]:
# ============================================================
# CELL 4: KHỞI CHẠY FLASK SERVER VỚI NGROK
# ============================================================

NGROK_AUTH_TOKEN = "     "  

# ==================== FLASK APP ====================
app = Flask(__name__)
CORS(app)

@app.route('/api/health', methods=['GET'])
def health_check():
    """Kiểm tra trạng thái server và models"""
    return jsonify({
        "status": "ready",
        "message": "OCR Backend is ready - Models pre-loaded",
        "models": {
            "llama": "loaded",
            "craft": "loaded",
            "tesseract": "loaded"
        }
    })

@app.route('/api/ocr', methods=['POST'])
def process_ocr():
    """API endpoint xử lý OCR"""
    try:
        if 'image' not in request.files:
            return jsonify({"success": False, "error": "No image file"}), 400

        file = request.files['image']
        if file.filename == '':
            return jsonify({"success": False, "error": "No file selected"}), 400

        print(f"\n{'='*50}")
        print(f" Received: {file.filename}")

        image_bytes = file.read()
        result = process_image_api(image_bytes)

        if "error" in result:
            return jsonify({"success": False, "error": result["error"]}), 500

        return jsonify({"success": True, "data": result})

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({"success": False, "error": str(e)}), 500

@app.route('/api/ocr/base64', methods=['POST'])
def process_ocr_base64():
    """API cho ảnh base64"""
    try:
        data = request.get_json()
        if 'image' not in data:
            return jsonify({"success": False, "error": "No image data"}), 400

        image_data = data['image']
        if ',' in image_data:
            image_data = image_data.split(',')[1]

        image_bytes = base64.b64decode(image_data)
        result = process_image_api(image_bytes)

        if "error" in result:
            return jsonify({"success": False, "error": result["error"]}), 500

        return jsonify({"success": True, "data": result})

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({"success": False, "error": str(e)}), 500

# ==================== KHỞI CHẠY NGROK ====================
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

try:
    ngrok.kill()
except:
    pass

public_url = ngrok.connect(5000)
public_url_str = str(public_url).replace("NgrokTunnel: ", "").split(" ->")[0].strip('"')

print("\n" + "=" * 70)
print(" SERVER ĐÃ SẴN SÀNG - MODELS ĐÃ LOAD TRƯỚC!")
print("=" * 70)
print(f" URL: {public_url_str}")
print(f" Health: {public_url_str}/api/health")
print(f" OCR API: {public_url_str}/api/ocr")
print("=" * 70)
print(" Xử lý sẽ NHANH vì models đã được load sẵn!")
print(" Copy URL trên và paste vào Frontend")
print("=" * 70)

# ==================== RUN SERVER ====================
def run_server():
    app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False)

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()

print("\n Server đang chạy! Đừng tắt notebook này.")
print(" Thời gian xử lý ảnh: ~5-15 giây (thay vì 30-60 giây)")


 SERVER ĐÃ SẴN SÀNG - MODELS ĐÃ LOAD TRƯỚC!
 URL: https://fidel-euchromatic-sheryl.ngrok-free.dev
 Health: https://fidel-euchromatic-sheryl.ngrok-free.dev/api/health
 OCR API: https://fidel-euchromatic-sheryl.ngrok-free.dev/api/ocr
 Xử lý sẽ NHANH vì models đã được load sẵn!
 Copy URL trên và paste vào Frontend

 Server đang chạy! Đừng tắt notebook này.
 Thời gian xử lý ảnh: ~5-15 giây (thay vì 30-60 giây)


#